In [1]:
#setup based on this: https://t-redactyl.io/blog/2020/08/reading-s3-data-into-a-spark-dataframe-using-sagemaker.html
import boto3
import json 
import time
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import sagemaker_pyspark
import botocore.session

## Set Spark Session Configuration

In [2]:
session = botocore.session.get_session()
credentials = session.get_credentials()

In [3]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(
    SecretId='sapient-s3-access'
)
response = json.loads(response['SecretString'])
access_key = response["aws_access_key_id"]
secret_key = response["aws_secret_access_key"]

In [4]:
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars()))
       )

In [5]:
# https://spark.apache.org/docs/latest/configuration.html#memory-management
spark = (
    SparkSession
    .builder
    .config(conf=conf) \
    .config('fs.s3a.access.key', access_key)
    .config('fs.s3a.secret.key', secret_key)
    .config('spark.network.timeout', 300)
    .config('spark.local.dir', '/home/ec2-user/SageMaker/tmp')
    .config("spark.executor.memory", "400g")
    .config("spark.driver.memory", "200g")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size","50g")
    .appName("sapient")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 05:21:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/15 05:21:42 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


## Functions to Load and Read Data

In [6]:
# read from raw bucket + write to refined bucket + aggregate final to the trusted bucket
s3_url_raw = "s3a://sapient-bucket-raw/"
s3_url_refined = "s3a://sapient-bucket-refined/"
s3_url_trusted = "s3a://sapient-bucket-trusted/"
ecar_cols = [
    'id','timestamp','objectID','actorID','object','action','hostname', 'user_name', 'privileges', 'image_path', 
    'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user'
            ]
bro_cols_conn = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 
                 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents']
bro_cols_rep = ['ts', 'level', 'message', 'location']

In [7]:
# Create a new dataframe with distinct objectIDs to identify malcious ObjectIds
df_labels = spark.read.parquet(f"{s3_url_refined}/prod/labels").cache()
df_malcious_objectIDs = df_labels.select('id').distinct()
df_labels.unpersist()

DataFrame[hostname: string, id: string, objectID: string, actorID: string, timestamp: timestamp, object: string, action: string]

In [8]:
def readCheckpoint(type='ecar', env='prod', size='small'):
    """
    type: ecar, ecar-bro, bro
    """
    if type == 'labels':
        s3_parquet_loc = f"{s3_url_trusted}/{env}/{type}"
    else:
        s3_parquet_loc = f"{s3_url_trusted}/{env}/{type}/{size}"
    start_time = time.time()
    df = spark.read.parquet(s3_parquet_loc).cache()
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))
    return df

In [9]:
def get_firsts(df):
    """
    this creates then drops duplicates and gets the first appearance of each relationship entry
    input - dataframe with columns objectID and actorID
    output dataframe
    """
    window = Window.partitionBy("relationship").orderBy("timestamp")
    df_new = df.withColumn('relationship', concat(df.actorID, lit('->'),df.objectID) ) \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1) \
                .drop('rank')
    return df_new

In [10]:
def write_firsts(size='all'):
    """
    this creates then drops duplicates and writes them to file in S3
    """
    start_time = time.time()
    df = readCheckpoint(size=size).cache()
    df_first_events = get_firsts(df)
    df_first_events.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/first_events")
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and write time: %s seconds ---" % (time.time() - start_time))

In [11]:
df = readCheckpoint().cache()

 5:21AM UTC on Mar 15, 2023 --- read and cache time: 0.4112553596496582 seconds ---


In [ ]:
df.count()

In [12]:
df.limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,...,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,event_minute,event_day,event_hour,malicious
0,d543ece4-fab7-474e-b21b-085cc84c7a51,2019-09-23 18:00:00.012,444cc620-225e-4ee3-89bc-b0157e0164b9,77727d87-de39-415e-8845-c114f010df0d,FLOW,INFO,SysClient0918.systemia.com,None,None,None,...,None,None,None,None,None,None,0,23,18,0
1,e421ef19-1bdf-48ad-9d0c-8d6a8e8564c5,2019-09-23 18:00:00.029,732f0dbd-56e7-4812-9e04-45df4f97394f,77727d87-de39-415e-8845-c114f010df0d,FLOW,INFO,SysClient0918.systemia.com,None,None,None,...,None,None,None,None,None,None,0,23,18,0
2,c4bd67c9-8f52-44c6-b2a2-9e36b8a52436,2019-09-23 18:00:00.035,2df0b3f4-dc61-425a-91fb-29f07edcfb7b,77727d87-de39-415e-8845-c114f010df0d,FLOW,INFO,SysClient0918.systemia.com,None,None,None,...,None,None,None,None,None,None,0,23,18,0
3,37005f78-273a-42d0-9b4c-e9c153b23b41,2019-09-23 18:00:00.045,c5a78b45-6712-44ff-81ff-cc3d26311860,77727d87-de39-415e-8845-c114f010df0d,FLOW,INFO,SysClient0918.systemia.com,None,None,None,...,None,None,None,None,None,None,0,23,18,0
4,d408ae4e-01a8-47e4-97b7-5625311298fc,2019-09-23 18:00:00.088,fd927524-290b-4618-bda6-2d54b97d688f,77727d87-de39-415e-8845-c114f010df0d,FLOW,INFO,SysClient0918.systemia.com,None,None,None,...,None,None,None,None,None,None,0,23,18,0


In [13]:
df_first_events = get_firsts(df).cache()

In [14]:
df.unpersist()

DataFrame[id: string, timestamp: timestamp, objectID: string, actorID: string, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, event_minute: int, event_day: int, event_hour: int, malicious: int]

In [15]:
df_first_events.limit(5).toPandas()

23/03/15 05:20:02 ERROR Executor: Exception in task 70.0 in stage 3.0 (TID 73)
java.io.FileNotFoundException: /home/ec2-user/SageMaker/tmp/blockmgr-17f6b4be-a92a-47cc-99d5-c260c1d6a507/14/temp_shuffle_30353522-586c-45c2-aa8e-54487bd3dd7a (Too many open files)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:140)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:159)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.s

Py4JJavaError: An error occurred while calling o99.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 49 in stage 3.0 failed 1 times, most recent failure: Lost task 49.0 in stage 3.0 (TID 52) (ip-172-16-0-97.us-west-2.compute.internal executor driver): java.io.FileNotFoundException: /home/ec2-user/SageMaker/tmp/blockmgr-17f6b4be-a92a-47cc-99d5-c260c1d6a507/2b/temp_shuffle_2c3b9792-364b-47b6-80e4-1276468f2e9d (Too many open files)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:140)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:159)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMer

In [ ]:
df_first_events.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/first_events")